In [17]:
import gradio as gr
import librosa
import numpy as np
from transformers import pipeline
import requests
import json
import textwrap

In [18]:
OPENROUTER_API_KEY = "sk-or-v1-ab7fdbd68c20ab2bf782087b29a5354ec96b02"
asr_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-small")

def transcribe_audio(audio_file):
    audio, sr = librosa.load(audio_file, sr=16000)
    if len(audio) > 480000:
        result = asr_pipeline(audio.astype(np.float32), return_timestamps=True)
    else:
        result = asr_pipeline(audio.astype(np.float32))
    return result["text"]

c:\Users\erand\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
def summarize_with_openrouter(transcript):
    try:
        prompt = (
            f"Please summarize the following meeting transcript in a detailed yet concise format."
            f"Highlight key decisions, action items, important discussions, and next steps.\n\n"
            f"Transcript:\n{transcript}"
        )
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",  # API endpoint
            headers={
                "Authorization": f"Bearer {OPENROUTER_API_KEY}",  # Pass the API key
                "Content-Type": "application/json",  # Indicate JSON content
                "HTTP-Referer": "https://yourdomain.com",  # Optional referer header
                "X-Title": "AI Meeting Summarizer",  # Optional request title
            },
            data=json.dumps({  # Convert the request body to JSON
                "model": "deepseek/deepseek-chat:free",  # Use the free DeepSeek Chat model
                "messages": [
                    {
                        "role": "user",  # User role message (typical for Chat API)
                        "content": prompt  # Include the custom prompt
                    }
                ],
            })
        )
        if response.status_code == 200:
            summary = response.json()['choices'][0]['message']['content']
            return textwrap.fill(summary, width=100)
        else:
            return f"Failed to generate summary. API returned status: {response.status_code}"
    except Exception as e:
        return f"An error occured during summarization: {e}"

In [20]:
def transcribe_and_summarize(audio_file):
    try:
        transcript = transcribe_audio(audio_file)
        summary = summarize_with_openrouter(transcript)
        return transcript, summary
    except Exception as e:
        return f"Error: {str(e)}", ""

iface = gr.Interface(
    fn = transcribe_and_summarize,
    inputs = gr.Audio(type="filepath", label="Upload MP3 File"),
    outputs = [
        gr.Textbox(label="Meeting Transcript"),
        gr.Textbox(label="AI Generated Summary")
    ],
    title = "AI Meeting Transcriber & Summarizer",
    description = "Upload a meeting recording. This tools will transcribe and summarize it using Whisper and DeepSeek"
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
